# Mixed formulation for the Poisson equation

This demo illustrates how to solve Poisson equation using a mixed
(two-field) formulation. In particular, it illustrates how to

* Use mixed and non-continuous finite element spaces.
* Set essential boundary conditions for subspaces and $H(\mathrm{div})$ spaces.


```{admonition} Download sources
:class: download

* {download}`Python script <./demo_mixed-poisson.py>`
* {download}`Jupyter notebook <./demo_mixed-poisson.ipynb>`
```

## Equation and problem definition

An alternative formulation of Poisson equation can be formulated by
introducing an additional (vector) variable, namely the (negative)
flux: $\sigma = \nabla u$. The partial differential equations
then read

$$
\begin{align}
  \sigma - \nabla u &= 0 \quad {\rm in} \ \Omega, \\
  \nabla \cdot \sigma &= - f \quad {\rm in} \ \Omega,
\end{align}
$$
with boundary conditions

$$
  u = u_0 \quad {\rm on} \ \Gamma_{D},  \\
  \sigma \cdot n = g \quad {\rm on} \ \Gamma_{N}.
$$

The same equations arise in connection with flow in porous media, and are
also referred to as Darcy flow. Here $n$ denotes the outward pointing normal
vector on the boundary. Looking at the variational form, we see that the
boundary condition for the flux ($\sigma \cdot n = g$) is now an essential
boundary condition (which should be enforced in the function space), while
the other boundary condition ($u = u_0$) is a natural boundary condition
(which should be applied to the variational form). Inserting the boundary
conditions, this variational problem can be phrased in the general form: find
$(\sigma, u) \in \Sigma_g \times V$ such that

$$
   a((\sigma, u), (\tau, v)) = L((\tau, v))
   \quad \forall \ (\tau, v) \in \Sigma_0 \times V,
$$

where the variational forms $a$ and $L$ are defined as

$$
\begin{align}
  a((\sigma, u), (\tau, v)) &=
    \int_{\Omega} \sigma \cdot \tau + \nabla \cdot \tau \ u
  + \nabla \cdot \sigma \ v \ {\rm d} x, \\
  L((\tau, v)) &= - \int_{\Omega} f v \ {\rm d} x
  + \int_{\Gamma_D} u_0 \tau \cdot n  \ {\rm d} s,
\end{align}
$$
and $\Sigma_g = \{ \tau \in H({\rm div})$ such that $\tau \cdot n|_{\Gamma_N}
= g \}$ and $V = L^2(\Omega)$.

To discretize the above formulation, two discrete function spaces $\Sigma_h
\subset \Sigma$ and $V_h \subset V$ are needed to form a mixed function space
$\Sigma_h \times V_h$. A stable choice of finite element spaces is to let
$\Sigma_h$ be the Brezzi-Douglas-Marini elements of polynomial order
$k$ and let $V_h$ be discontinuous elements of polynomial order $k-1$.

We will use the same definitions of functions and boundaries as in the
demo for {doc}`the Poisson equation <demo_poisson>`. These are:

* $\Omega = [0,1] \times [0,1]$ (a unit square)
* $\Gamma_{D} = \{(0, y) \cup (1, y) \in \partial \Omega\}$
* $\Gamma_{N} = \{(x, 0) \cup (x, 1) \in \partial \Omega\}$
* $u_0 = 0$
* $g = \sin(5x)$   (flux)
* $f = 10\exp(-((x - 0.5)^2 + (y - 0.5)^2) / 0.02)$  (source term)

## Implementation

In [18]:

try:
    from petsc4py import PETSc

    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI

import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from ufl import Measure, SpatialCoordinate, TestFunctions, TrialFunctions, div, exp, inner

msh = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

k = 1
Q_el = element("BDMCF", msh.basix_cell(), k, dtype=default_real_type)
P_el = element("DG", msh.basix_cell(), k - 1, dtype=default_real_type)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(msh, V_el)

(sigma, u) = TrialFunctions(V)
(tau, v) = TestFunctions(V)

x = SpatialCoordinate(msh)
f = 10.0 * exp(-((x[0] - 0.5) * (x[0] - 0.5) + (x[1] - 0.5) * (x[1] - 0.5)) / 0.02)

dx = Measure("dx", msh)
a = inner(sigma, tau) * dx + inner(u, div(tau)) * dx + inner(div(sigma), v) * dx
L = -inner(f, v) * dx

# Get subspace of V
V0 = V.sub(0)

fdim = msh.topology.dim - 1
facets_top = mesh.locate_entities_boundary(msh, fdim, lambda x: np.isclose(x[1], 1.0))
Q, _ = V0.collapse()
dofs_top = fem.locate_dofs_topological((V0, Q), fdim, facets_top)


def f1(x):
    values = np.zeros((2, x.shape[1]))
    values[1, :] = np.sin(5 * x[0])
    return values


f_h1 = fem.Function(Q)
f_h1.interpolate(f1)
bc_top = fem.dirichletbc(f_h1, dofs_top, V0)


facets_bottom = mesh.locate_entities_boundary(msh, fdim, lambda x: np.isclose(x[1], 0.0))
dofs_bottom = fem.locate_dofs_topological((V0, Q), fdim, facets_bottom)


def f2(x):
    values = np.zeros((2, x.shape[1]))
    values[1, :] = -np.sin(5 * x[0])
    return values


f_h2 = fem.Function(Q)
f_h2.interpolate(f2)
bc_bottom = fem.dirichletbc(f_h2, dofs_bottom, V0)


bcs = [bc_top, bc_bottom]

problem = LinearProblem(
    a,
    L,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="coupled_poisson"
)
try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

sigma_h, u_h = w_h.split()

with io.XDMFFile(msh.comm, "out_mixed_poisson/u.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(u_h)

# from dolfinx.io import VTKFile
# with VTKFile(MPI.COMM_WORLD, "out_mixed_poisson/p.pvd", "w") as vtk:
#     vtk.write_mesh(msh)
#     vtk.write_function(u_h)

In [19]:
#example above using cg
try:
    from petsc4py import PETSc
    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        raise SystemExit
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    raise SystemExit

from mpi4py import MPI
import numpy as np

import ufl
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem

# ---------------------------------------------------------------------
# Mesh
# ---------------------------------------------------------------------
msh = mesh.create_unit_square(
    MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral
)

# ---------------------------------------------------------------------
# CG1 function space
# ---------------------------------------------------------------------
V = fem.functionspace(msh, ("Lagrange", 1))  # CG1

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(msh)

# RHS as in your mixed formulation
f = 10.0 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)

# Neumann data g = sin(5 x) on top and bottom
g = ufl.sin(5.0 * x[0])

dx = ufl.dx
ds = ufl.ds

# ---------------------------------------------------------------------
# Variational problem: standard Poisson with mixed Dirichlet/Neumann BCs
# ---------------------------------------------------------------------
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
L = f * v * dx + g * v * ds  # Neumann on horizontal edges

# ---------------------------------------------------------------------
# Dirichlet BC: u = 0 on left and right boundaries (x = 0, x = 1)
# ---------------------------------------------------------------------
tdim = msh.topology.dim
fdim = tdim - 1

facets_D = mesh.locate_entities_boundary(
    msh, fdim, lambda x_: np.isclose(x_[0], 0.0) | np.isclose(x_[0], 1.0)
)

dofs_D = fem.locate_dofs_topological(V, fdim, facets_D)
u_D = default_real_type(0.0)
bc = fem.dirichletbc(u_D, dofs_D, V)

# ---------------------------------------------------------------------
# Solve linear system with CG1
# ---------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
        "ksp_error_if_not_converged": True,
    },
    petsc_options_prefix="poisson_cg1_",
)

try:
    u_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        raise SystemExit
    else:
        raise e

# ---------------------------------------------------------------------
# Output
# ---------------------------------------------------------------------
with io.XDMFFile(msh.comm, "out_poisson_cg1/u.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(u_h)


In [ ]:
#mixed fem 2d only dirichlet bc on left right
try:
    from petsc4py import PETSc

    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI

import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from ufl import Measure, SpatialCoordinate, TestFunctions, TrialFunctions, div, exp, inner
import ufl

msh = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

k = 2
# Q_el = element("BDMCF", msh.basix_cell(), k, dtype=default_real_type)
Q_el = element("RTCF", msh.basix_cell(), k, dtype=default_real_type)
P_el = element("DG", msh.basix_cell(), k - 1, dtype=default_real_type)
# P_el = element("Lagrange", msh.basix_cell(), k, dtype=default_real_type)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(msh, V_el)

(u_m, p_m) = TrialFunctions(V)
(tau, v) = TestFunctions(V)

n = ufl.FacetNormal(msh)

f = fem.Constant(msh, 0.0)

dx = ufl.Measure("dx", msh)
ds_Omega = ufl.Measure("ds", domain=msh)

g_left  = 1.0
g_right = 4.0

xmin, xmax = 0.0, 1.0
tol = 1e-10 * (xmax - xmin)

x_ufl = ufl.SpatialCoordinate(msh)

g_D = ufl.conditional(
    ufl.lt(abs(x_ufl[0] - xmin), tol), g_left,
    ufl.conditional(
        ufl.lt(abs(x_ufl[0] - xmax), tol), g_right,
        0.0
    )
)

a = ufl.inner(u_m, tau) * dx - ufl.inner(p_m, ufl.div(tau)) * dx + ufl.inner(ufl.div(u_m), v) * dx
L = -ufl.inner(f, v) * dx - g_D * ufl.dot(tau, n) * ds_Omega

problem = LinearProblem(
    a,
    L,
    # bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="coupled_poisson"
)

try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_m_h, p_m_h = w_h.split()

# with io.XDMFFile(msh.comm, "out_mixed_poisson/p_m_2d_BDM_cg.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(p_m_h)

# Visualization space: continuous Lagrange P1
V_vis = fem.functionspace(msh, ("Lagrange", 1))
p_m_vis = fem.Function(V_vis)
p_m_vis.name = "p_m"

# Interpolate mixed-DG1 pressure into CG1
p_m_vis.interpolate(p_m_h)   # p_m is your mixed solution sub-function

with io.XDMFFile(msh.comm, "out_mixed_poisson/p_m_2d_BDM.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(p_m_vis)

# If you also want to write u_m:
# with io.XDMFFile(msh.comm, "out_mixed_poisson/u_m.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(u_m_h)


In [10]:
# mixed fem 2d use dirichlet and neumann bc
try:
    from petsc4py import PETSc
    import dolfinx
    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI
import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import locate_entities_boundary
from ufl import TestFunctions, TrialFunctions
import ufl

# ---------------------------------------------------------------------------
# Mesh
# ---------------------------------------------------------------------------
msh = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

# ---------------------------------------------------------------------------
# Mixed function space: flux (H(div)) x pressure (DG)
# ---------------------------------------------------------------------------
k = 2
Q_el = element("RTCF", msh.basix_cell(), k, dtype=default_real_type)     # flux space (H(div))
P_el = element("Lagrange", msh.basix_cell(), k - 1, dtype=default_real_type)   # pressure space (DG)
# P_el = element("DG", msh.basix_cell(), k-1, dtype=default_real_type)   # pressure space (DG)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(msh, V_el)

(u_m, p_m) = TrialFunctions(V)
(tau, v) = TestFunctions(V)

n = ufl.FacetNormal(msh)

f = fem.Constant(msh, default_real_type(0.0))

dx = ufl.Measure("dx", msh)
ds_Omega = ufl.Measure("ds", domain=msh)

# ---------------------------------------------------------------------------
# Dirichlet BC for pressure on left and right (weakly, via mixed formulation)
# ---------------------------------------------------------------------------
g_left  = default_real_type(1.0)
g_right = default_real_type(4.0)

xmin, xmax = 0.0, 1.0
tol = 1e-10 * (xmax - xmin)

x_ufl = ufl.SpatialCoordinate(msh)

# g_D: Dirichlet data for p on left and right
g_D = ufl.conditional(
    ufl.lt(abs(x_ufl[0] - xmin), tol), g_left,
    ufl.conditional(
        ufl.lt(abs(x_ufl[0] - xmax), tol), g_right,
        0.0
    )
)

# ---------------------------------------------------------------------------
# Dirichlet BC for flux on top and bottom (u·n prescribed)
# Top (y = 1):  u·n = -1  (n = (0,1) -> u = (0, -1))
# Bottom (y = 0): u·n =  2 (n = (0,-1) -> u = (0, -2))
# ---------------------------------------------------------------------------
fdim = msh.topology.dim - 1

def top_boundary(x):
    return np.isclose(x[1], 1.0)

def bottom_boundary(x):
    return np.isclose(x[1], 0.0)

top_facets = locate_entities_boundary(msh, fdim, top_boundary)
bottom_facets = locate_entities_boundary(msh, fdim, bottom_boundary)

# Flux subspace in the mixed space
Q_sub = V.sub(0)

# Collapse flux subspace to an independent space Q
Q, submap = Q_sub.collapse()

# Functions for flux values on top and bottom
u_top = fem.Function(Q)
u_bottom = fem.Function(Q)

# Set them to constants: top -> (0, -1), bottom -> (0, -2)
def top_flux_expr(x):
    # shape (dim, num_points)
    return np.vstack((
        np.zeros_like(x[0]),
        -np.ones_like(x[0])
    ))

def bottom_flux_expr(x):
    return np.vstack((
        np.zeros_like(x[0]),
        -2.0 * np.ones_like(x[0])
    ))

u_top.interpolate(top_flux_expr)
u_bottom.interpolate(bottom_flux_expr)

# Map facets to dofs in V.sub(0) via Q
top_dofs = fem.locate_dofs_topological((Q_sub, Q), fdim, top_facets)
bottom_dofs = fem.locate_dofs_topological((Q_sub, Q), fdim, bottom_facets)

bc_top = fem.dirichletbc(u_top, top_dofs, Q_sub)
bc_bottom = fem.dirichletbc(u_bottom, bottom_dofs, Q_sub)
bcs = [bc_top, bc_bottom]

# ---------------------------------------------------------------------------
# Variational formulation (mixed Poisson)
# ---------------------------------------------------------------------------
a = (
    ufl.inner(u_m, tau) * dx
    - ufl.inner(p_m, ufl.div(tau)) * dx
    + ufl.inner(ufl.div(u_m), v) * dx
)

# Note: g_D here is Dirichlet data for p on left/right (weakly enforced)
L = -ufl.inner(f, v) * dx - g_D * ufl.dot(tau, n) * ds_Omega

# ---------------------------------------------------------------------------
# Linear problem
# ---------------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="coupled_poisson"
)

try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_m_h, p_m_h = w_h.split()

# ---------------------------------------------------------------------------
# Postprocessing: project pressure to CG1 for visualization
# ---------------------------------------------------------------------------
# V_vis = fem.functionspace(msh, ("Lagrange", 1))
# p_m_vis = fem.Function(V_vis)
# p_m_vis.name = "p_m"

# p_m_vis.interpolate(p_m_h)

# with io.XDMFFile(msh.comm, "out_mixed_poisson/p_m_2d_mixed_RTcg.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(p_m_vis)

with io.XDMFFile(msh.comm, "out_mixed_poisson/p_m_2d_mixed_RTcg.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(p_m_h)

# If you also want to write u_m:
# with io.XDMFFile(msh.comm, "out_mixed_poisson/u_m_2d_mixed.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(u_m_h)


In [6]:
# Standard Poisson CG1 formulation with Dirichlet p (left/right)
# and flux (Neumann) BC on top and bottom.

try:
    from petsc4py import PETSc
    import dolfinx
    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        raise SystemExit
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    raise SystemExit

from mpi4py import MPI
import numpy as np

from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
import ufl

# ---------------------------------------------------------------------------
# Mesh
# ---------------------------------------------------------------------------
msh = mesh.create_unit_square(
    MPI.COMM_WORLD, 32, 32, cell_type=mesh.CellType.quadrilateral
)

# ---------------------------------------------------------------------------
# Function space: scalar CG1 for pressure p
# ---------------------------------------------------------------------------
V = fem.functionspace(msh, ("Lagrange", 1))
p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

dx = ufl.Measure("dx", domain=msh)
ds = ufl.Measure("ds", domain=msh)

# ---------------------------------------------------------------------------
# RHS / source term
# ---------------------------------------------------------------------------
f = fem.Constant(msh, default_real_type(0.0))  # same as your mixed example

# ---------------------------------------------------------------------------
# Dirichlet BC: p on left and right
# ---------------------------------------------------------------------------
g_left = default_real_type(1.0)
g_right = default_real_type(4.0)

xmin, xmax = 0.0, 1.0

def left_boundary(x):
    return np.isclose(x[0], xmin)

def right_boundary(x):
    return np.isclose(x[0], xmax)

left_dofs = fem.locate_dofs_geometrical(V, left_boundary)
right_dofs = fem.locate_dofs_geometrical(V, right_boundary)

bc_left = fem.dirichletbc(g_left, left_dofs, V)
bc_right = fem.dirichletbc(g_right, right_dofs, V)
bcs = [bc_left, bc_right]

# ---------------------------------------------------------------------------
# Neumann BC: flux on top and bottom
#
# Mixed form had u = -grad p, u·n given as:
#   top (y=1):    u·n = -1
#   bottom (y=0): u·n =  2
#
# For -Δp = f, natural BC is ∂p/∂n = g_N.
# Since u = -grad p, we have ∂p/∂n = -u·n:
#   top:    ∂p/∂n =  1
#   bottom: ∂p/∂n = -2
# ---------------------------------------------------------------------------
tol = 1e-10
x = ufl.SpatialCoordinate(msh)

g_N = ufl.conditional(
    ufl.lt(abs(x[1] - 1.0), tol),
    default_real_type(1.0),  # top: ∂p/∂n = 1
    ufl.conditional(
        ufl.lt(abs(x[1] - 0.0), tol),
        default_real_type(-2.0),  # bottom: ∂p/∂n = -2
        default_real_type(0.0)
    )
)

# ---------------------------------------------------------------------------
# Variational problem: -Δp = f
#
# ∫Ω grad p · grad v dx = ∫Ω f v dx + ∫Γ_N g_N v ds
# ---------------------------------------------------------------------------
a = ufl.inner(ufl.grad(p), ufl.grad(v)) * dx
L = f * v * dx + g_N * v * ds

# ---------------------------------------------------------------------------
# Solve
# ---------------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="poisson_cg1",
)

try:
    p_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        raise SystemExit
    else:
        raise e

p_h.name = "p"

# ---------------------------------------------------------------------------
# Output
# ---------------------------------------------------------------------------
with io.XDMFFile(msh.comm, "out_poisson_mixed/p_2d_cg1.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(p_h)


In [11]:
# mixed fem 2d NonlinearProblem
try:
    from petsc4py import PETSc
    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI
import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem import petsc  # <- use petsc.NonlinearProblem
from ufl import Measure, SpatialCoordinate, TestFunctions, div, inner
import ufl

# -------------------------------------------------------------------------
# Mesh and spaces
# -------------------------------------------------------------------------
msh = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

k = 1
Q_el = element("BDMCF", msh.basix_cell(), k, dtype=default_real_type)
# P_el = element("DG", msh.basix_cell(), k - 1, dtype=default_real_type)
P_el = element("Lagrange", msh.basix_cell(), k, dtype=default_real_type)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(msh, V_el)

# Unknown mixed function and test functions
w = fem.Function(V)          # holds (u_m, p_m)
(u_m, p_m) = ufl.split(w)
(tau, v) = TestFunctions(V)

n = ufl.FacetNormal(msh)

f = fem.Constant(msh, default_real_type(0.0))

dx = ufl.Measure("dx", msh)
ds_Omega = ufl.Measure("ds", domain=msh)

# -------------------------------------------------------------------------
# Boundary data
# -------------------------------------------------------------------------
g_left  = 1.0
g_right = 4.0

xmin, xmax = 0.0, 1.0
tol = 1e-10 * (xmax - xmin)

x_ufl = ufl.SpatialCoordinate(msh)

g_D = ufl.conditional(
    ufl.lt(abs(x_ufl[0] - xmin), tol), g_left,
    ufl.conditional(
        ufl.lt(abs(x_ufl[0] - xmax), tol), g_right,
        0.0
    )
)

# -------------------------------------------------------------------------
# Variational forms
# a(u,p; tau, v) = L(tau, v)  ->  F = a - L = 0
# -------------------------------------------------------------------------
a = inner(u_m, tau) * dx - inner(p_m, div(tau)) * dx + inner(div(u_m), v) * dx
L = -inner(f, v) * dx - g_D * ufl.dot(tau, n) * ds_Omega

F = a - L                     # residual
J = ufl.derivative(F, w)      # Jacobian

# No Dirichlet BCs in this formulation
bcs = []

# -------------------------------------------------------------------------
# NonlinearProblem with PETSc SNES
# -------------------------------------------------------------------------
nlp = petsc.NonlinearProblem(
    F,
    u=w,
    J=J,
    bcs=bcs,
    petsc_options={
        "snes_monitor": None,
        "snes_max_it": 200,  # can adjust
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
        "ksp_error_if_not_converged": True,
        "snes_error_if_not_converged": True,
    },
    petsc_options_prefix="coupled_poisson_",
)

# Initial guess
w.x.array[:] = 0.0

try:
    w_h = nlp.solve()  # updates w in-place and returns it
    iterations = nlp.solver.getIterationNumber()
    print(f"Converged in {iterations} Newton iterations")
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# -------------------------------------------------------------------------
# Post-processing
# -------------------------------------------------------------------------
u_m_h, p_m_h = w_h.split()

with io.XDMFFile(msh.comm, "out_mixed_poisson/p_m_2d_BDM_cg_nonlinear.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(p_m_h)

# If you also want to write u_m:
# with io.XDMFFile(msh.comm, "out_mixed_poisson/u_m.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(u_m_h)


  0 SNES Function norm 2.332380757938e+01
  1 SNES Function norm 2.327966277468e+01
  2 SNES Function norm 2.327963813504e+01
  3 SNES Function norm 2.327156106120e+01
  4 SNES Function norm 2.324640405153e+01
  5 SNES Function norm 2.323980921810e+01
  6 SNES Function norm 2.323753147426e+01


Error: error code 91
[0] SNESSolve() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1759674433/work/src/snes/interface/snes.c:4906
[0] SNESSolve_NEWTONLS() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1759674433/work/src/snes/impls/ls/ls.c:253
[0] SNESSolve has not converged due to DIVERGED_LINE_SEARCH. Suggest running with -snes_linesearch_monitor

In [ ]:
#Mixed fem in 3d, using above code as inspiration (strong bc)
try:
    from petsc4py import PETSc
    import dolfinx
    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        raise SystemExit
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    raise SystemExit

from mpi4py import MPI
import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import gmsh as gmshio
from ufl import (Measure, SpatialCoordinate, TestFunctions, TrialFunctions,
                 div, exp, inner)

# -------------------------------------------------------------------------
# 3D mesh from Gmsh
# -------------------------------------------------------------------------
filename = f"Omega_ex1_{3}.msh"
Omega, cell_tags_Omega, facet_tags_Omega = gmshio.read_from_msh(
    filename, MPI.COMM_WORLD, 0, gdim=3
)[0:3]

# -------------------------------------------------------------------------
# Function spaces (H(div) × DG) in 3D
#   u_m : flux (vector, H(div))
#   p_m : pressure (scalar, DG)
# -------------------------------------------------------------------------
k = 1
# Q_el = element("BDMCF", Omega.basix_cell(), k, dtype=default_real_type)   # vector, H(div)
Q_el = element("RTCF", Omega.basix_cell(), k-1, dtype=default_real_type)   # vector, H(div)
P_el = element("DG",    Omega.basix_cell(), k - 1, dtype=default_real_type)  # scalar
# P_el = element("Lagrange", Omega.basix_cell(), k, dtype=default_real_type)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(Omega, V_el)

(u_m, p_m) = TrialFunctions(V)
(v_m, q_m) = TestFunctions(V)

# -------------------------------------------------------------------------
# RHS (optional body force term)
# -------------------------------------------------------------------------
x = SpatialCoordinate(Omega)
f = fem.Constant(Omega, 0.0)

dx = Measure("dx", domain=Omega)

# -------------------------------------------------------------------------
# Tensor k_m (3×3 constant) and mixed formulation:
#
# a = ( (u_m, v_m)
#       - p_m * div(k_m * v_m)
#       + q_m * div(u_m) ) * dx
# -------------------------------------------------------------------------
k_values = np.array(
    [[1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0]],
    dtype=default_real_type
)
k_m = fem.Constant(Omega, k_values)

a = (
    inner(u_m, v_m)
    - p_m * div(k_m * v_m)
    + q_m * div(u_m)
) * dx

# Keep the same sign convention as your original example:
L = -f * q_m * dx

# -------------------------------------------------------------------------
# Dirichlet BCs: p_m = 1 on "left", p_m = 4 on "right"
#   We apply BCs on the pressure subspace (index 1)
# -------------------------------------------------------------------------
V_p = V.sub(1)          # pressure subspace
Q_p, _ = V_p.collapse() # scalar DG space

fdim = Omega.topology.dim - 1

# "Left": x[0] ≈ 0.0
facets_left = mesh.locate_entities_boundary(
    Omega, fdim, lambda x: np.isclose(x[0], 0.0)
)
dofs_left = fem.locate_dofs_topological((V_p, Q_p), fdim, facets_left)

def f1(x):
    values = np.zeros((1, x.shape[1])) + 1
    return values

p_left = fem.Function(Q_p)
p_left.interpolate(f1)
bc_left = fem.dirichletbc(p_left, dofs_left, V_p)

# "Right": x[0] ≈ 1.0
facets_right = mesh.locate_entities_boundary(
    Omega, fdim, lambda x: np.isclose(x[0], 1.0)
)
dofs_right = fem.locate_dofs_topological((V_p, Q_p), fdim, facets_right)

def f2(x):
    values = np.zeros((1, x.shape[1])) + 4
    return values

p_right = fem.Function(Q_p)
p_right.interpolate(f1)
# p_right.x.array[:] = 4.0
bc_right = fem.dirichletbc(p_right, dofs_right, V_p)

bcs = [bc_left, bc_right]

# -------------------------------------------------------------------------
# Linear solve
# -------------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="coupled_poisson"
)

try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        raise SystemExit
    else:
        raise e

u_m_h, p_m_h = w_h.split()

# -------------------------------------------------------------------------
# Output
# -------------------------------------------------------------------------
# with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/p_m.xdmf", "w") as file:
#     file.write_mesh(Omega)
#     file.write_function(p_m_h)

# Visualization space: continuous Lagrange P1
V_vis = fem.functionspace(Omega, ("Lagrange", 1))
p_m_vis = fem.Function(V_vis)
p_m_vis.name = "p_m"

# Interpolate mixed-DG1 pressure into CG1
p_m_vis.interpolate(p_m_h)   # p_m is your mixed solution sub-function

with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/p_m.xdmf", "w") as file:
    file.write_mesh(Omega)
    file.write_function(p_m_vis)

# with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/u_m.xdmf", "w") as file:
#     file.write_mesh(Omega)
#     file.write_function(u_m_h)


Info    : Reading 'Omega_ex1_3.msh'...
Info    : 27 entities
Info    : 578 nodes
Info    : 256 elements
Info    : Done reading 'Omega_ex1_3.msh'


In [8]:
#mixed fem in 3d using weak bc
try:
    from petsc4py import PETSc

    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI

import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import gmsh as gmshio
import ufl
from ufl import (
    Measure,
    SpatialCoordinate,
    TestFunctions,
    TrialFunctions,
    div,
    exp,
    inner,
)

# ---------------------------------------------------------------------
# 3D mesh from Gmsh
# ---------------------------------------------------------------------
filename = f"Omega_ex1_{3}.msh"
Omega, cell_tags_Omega, facet_tags_Omega = gmshio.read_from_msh(
    filename, MPI.COMM_WORLD, 0, gdim=3
)[0:3]
n = ufl.FacetNormal(Omega)

# ---------------------------------------------------------------------
# Function spaces and mixed variables
#   u_m : flux (H(div), vector)
#   p_m : pressure (DG, scalar)
# ---------------------------------------------------------------------
k = 1
Q_el = element("BDMCF", Omega.basix_cell(), k, dtype=default_real_type)
# P_el = element("DG", Omega.basix_cell(), k - 1, dtype=default_real_type)
P_el = element("Lagrange", Omega.basix_cell(), k, dtype=default_real_type)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(Omega, V_el)

(u_m, p_m) = TrialFunctions(V)
(v_m, q_m) = TestFunctions(V)

x = SpatialCoordinate(Omega)
f = fem.Constant(Omega, 0.0)

dx = Measure("dx", Omega)
ds_Omega = ufl.Measure("ds", domain=Omega)

# ---------------------------------------------------------------------
# k_m tensor and mixed formulation
#   a = ( (u_m, v_m)
#         - p_m * div(k_m * v_m)
#         + q_m * div(u_m) ) * dx
# ---------------------------------------------------------------------
k_values = np.array(
    [
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 0.00],
    ],
    dtype=default_real_type,
)
k_m = fem.Constant(Omega, k_values)

a = (
    inner(u_m, v_m)
    - p_m * div(k_m * v_m)
    + q_m * div(u_m)
) * dx

# ---------------------------------------------------------------------
# 4. Geometry and Dirichlet data g_D (left/right) – weak enforcement
# ---------------------------------------------------------------------
coords = Omega.geometry.x
x = coords[:, 0]
y = coords[:, 1]

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

tol = 1e-10 * max(xmax - xmin, ymax - ymin)

x_ufl = ufl.SpatialCoordinate(Omega)

g_left  = 1.0
g_right = 4.0

g_D = ufl.conditional(
    ufl.lt(abs(x_ufl[0] - xmin), tol), g_left,
    ufl.conditional(
        ufl.lt(abs(x_ufl[0] - xmax), tol), g_right,
        0.0
    )
)

# L = -f * q_m * dx
L = f * q_m * dx - g_D * ufl.dot(k_m * v_m, n) * ds_Omega

# ---------------------------------------------------------------------
# Boundary conditions: left p_m = 1, right p_m = 4
#   Follow the same style as your original example:
#   - locate_entities_boundary
#   - locate_dofs_topological
#   - dirichletbc on the pressure subspace
# ---------------------------------------------------------------------
# V_p = V.sub(1)            # subspace for p_m
# Q_p, _ = V_p.collapse()   # scalar DG space

# fdim = Omega.topology.dim - 1

# # "Left": x[0] ≈ 0.0
# facets_left = mesh.locate_entities_boundary(
#     Omega, fdim, lambda x: np.isclose(x[0], 0.0)
# )
# dofs_left = fem.locate_dofs_topological((V_p, Q_p), fdim, facets_left)

# p_left = fem.Function(Q_p)
# p_left.x.array[:] = 1.0
# bc_left = fem.dirichletbc(p_left, dofs_left, V_p)

# # "Right": x[0] ≈ 1.0
# facets_right = mesh.locate_entities_boundary(
#     Omega, fdim, lambda x: np.isclose(x[0], 1.0)
# )
# dofs_right = fem.locate_dofs_topological((V_p, Q_p), fdim, facets_right)

# p_right = fem.Function(Q_p)
# p_right.x.array[:] = 4.0
# bc_right = fem.dirichletbc(p_right, dofs_right, V_p)

# bcs = [bc_left, bc_right]

# Bounding box of the geometry
# coords = Omega.geometry.x
# x_coords = coords[:, 0]
# y_coords = coords[:, 1]
# z_coords = coords[:, 2]

# xmin, xmax = x_coords.min(), x_coords.max()
# ymin, ymax = y_coords.min(), y_coords.max()
# zmin, zmax = z_coords.min(), z_coords.max()

# # Tolerance for side detection (3D)
# tol = 1e-10 * max(xmax - xmin, ymax - ymin, zmax - zmin)

# # Locate dofs on left/right using geometry
# left_dofs = fem.locate_dofs_geometrical(
#     Q_p, lambda x: np.isclose(x[0], xmin, atol=tol)
# )
# right_dofs = fem.locate_dofs_geometrical(
#     Q_p, lambda x: np.isclose(x[0], xmax, atol=tol)
# )

# # Union of boundary dofs where we enforce p_m
# all_dofs = np.unique(np.concatenate([left_dofs, right_dofs]))

# # Piecewise Function-valued BC in the pressure subspace
# p_m_bc = fem.Function(Q_p)
# # default is 0 everywhere; set left/right:
# p_m_bc.x.array[left_dofs] = 1.0
# p_m_bc.x.array[right_dofs] = 4.0

# # Single BC object with piecewise values
# bc_pm = fem.dirichletbc(p_m_bc, all_dofs)

# bcs = [bc_pm]

# ---------------------------------------------------------------------
# Linear solve
# ---------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    # bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="coupled_poisson"
)
try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_m_h, p_m_h = w_h.split()

# ---------------------------------------------------------------------
# Output (same style as your example)
# ---------------------------------------------------------------------
with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/p_m_3d_BDM_cg.xdmf", "w") as file:
    file.write_mesh(Omega)
    file.write_function(p_m_h)

# with io.VTXWriter(Omega.comm, "out_mixed_poisson_3d/p_m.bp", [p_m_h]) as vtx:
#     vtx.write(0.0)

# Visualization space: continuous Lagrange P1
# V_vis = fem.functionspace(Omega, ("Lagrange", 1))
# p_m_vis = fem.Function(V_vis)
# p_m_vis.name = "p_m"

# # Interpolate mixed-DG1 pressure into CG1
# p_m_vis.interpolate(p_m_h)   # p_m is your mixed solution sub-function

# with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/p_m_3d_BDM.xdmf", "w") as file:
#     file.write_mesh(Omega)
#     file.write_function(p_m_vis)

# with io.XDMFFile(Omega.comm, "out_mixed_poisson_3d/u_m.xdmf", "w") as file:
#     file.write_mesh(Omega)
#     file.write_function(u_m_h)

# from dolfinx.io import VTKFile
# with VTKFile(MPI.COMM_WORLD, "out_mixed_poisson_3d/p.pvd", "w") as vtk:
#     vtk.write_mesh(Omega)
#     vtk.write_function(p_m_h)


Info    : Reading 'Omega_ex1_3.msh'...
Info    : 27 entities
Info    : 578 nodes
Info    : 256 elements
Info    : Done reading 'Omega_ex1_3.msh'


In [26]:
from dolfinx import mesh, fem, io
import ufl
from mpi4py import MPI
import numpy as np

# Example mesh and DG1 space
domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
V = fem.functionspace(domain, ("DG", 1))
u = fem.Function(V)
u.x.array[:] = np.random.rand(u.x.array.size)  # some DG1 data

# with io.XDMFFile(MPI.COMM_WORLD, "u_dg1.xdmf", "w") as xdmf:
#     xdmf.write_mesh(domain)
#     xdmf.write_function(u)

from dolfinx import io

# domain: dolfinx mesh
# u: fem.Function on DG1

with io.VTXWriter(domain.comm, "u_dg1.bp", [u]) as vtx:
    vtx.write(0.0)


In [30]:
from mpi4py import MPI
from dolfinx import mesh, fem, io
import numpy as np

domain = mesh.create_unit_square(MPI.COMM_WORLD, 4, 4)
V = fem.functionspace(domain, ("DG", 1))
u = fem.Function(V)
u.x.array[:] = np.random.rand(len(u.x.array))
u.name = "u_dg1"

with io.VTXWriter(domain.comm, "u_dg1.bp", [domain, u]) as vtx:
    vtx.write(0.0)

AttributeError: 'Mesh' object has no attribute 'function_space'

In [9]:
#CG
try:
    from petsc4py import PETSc

    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
except ModuleNotFoundError:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI

import numpy as np

from dolfinx import default_real_type, fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import gmsh as gmshio
import ufl

# ---------------------------------------------------------------------
# 1. 3D mesh from Gmsh
# ---------------------------------------------------------------------
filename = f"Omega_ex1_{3}.msh"
Omega, cell_tags_Omega, facet_tags_Omega = gmshio.read_from_msh(
    filename, MPI.COMM_WORLD, 0, gdim=3
)[0:3]

# ---------------------------------------------------------------------
# 2. Standard CG space and scalar variables
#    u : scalar potential (CG)
# ---------------------------------------------------------------------
k = 1  # polynomial degree
V = fem.functionspace(Omega, ("Lagrange", k))

p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

f = fem.Constant(Omega, default_real_type(0.0))

dx = ufl.Measure("dx", domain=Omega)

# Bilinear form and RHS:
#   a(u, v) = ∫ grad(u)·grad(v) dx
#   L(v)    = ∫ f v dx
a = ufl.dot(k_m*ufl.grad(p), ufl.grad(v)) * dx
L = f * v * dx

# ---------------------------------------------------------------------
# 3. Geometry and Dirichlet data: left/right x-boundaries
#    u = g_left on x = xmin
#    u = g_right on x = xmax
# ---------------------------------------------------------------------
coords = Omega.geometry.x
x_coords = coords[:, 0]
y_coords = coords[:, 1]
z_coords = coords[:, 2]

xmin, xmax = x_coords.min(), x_coords.max()
ymin, ymax = y_coords.min(), y_coords.max()
zmin, zmax = z_coords.min(), z_coords.max()

tol = 1e-10 * max(xmax - xmin, ymax - ymin, zmax - zmin)

g_left = 1.0
g_right = 4.0

fdim = Omega.topology.dim - 1

# "Left": x ≈ xmin
facets_left = mesh.locate_entities_boundary(
    Omega, fdim, lambda x: np.isclose(x[0], xmin, atol=tol)
)
# "Right": x ≈ xmax
facets_right = mesh.locate_entities_boundary(
    Omega, fdim, lambda x: np.isclose(x[0], xmax, atol=tol)
)

dofs_left = fem.locate_dofs_topological(V, fdim, facets_left)
dofs_right = fem.locate_dofs_topological(V, fdim, facets_right)

bc_left = fem.dirichletbc(PETSc.ScalarType(g_left), dofs_left, V)
bc_right = fem.dirichletbc(PETSc.ScalarType(g_right), dofs_right, V)

bcs = [bc_left, bc_right]

# ---------------------------------------------------------------------
# 4. Linear solve (standard CG Poisson)
# ---------------------------------------------------------------------
problem = LinearProblem(
    a,
    L,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "superlu_dist",
    },
    petsc_options_prefix="cg_poisson",
)

try:
    u_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# ---------------------------------------------------------------------
# 5. Output
# ---------------------------------------------------------------------
with io.XDMFFile(Omega.comm, "out_cg_poisson_3d/p_m_cg.xdmf", "w") as file:
    file.write_mesh(Omega)
    file.write_function(u_h)

# If you prefer VTK:
# from dolfinx.io import VTKFile
# with VTKFile(MPI.COMM_WORLD, "out_cg_poisson_3d/u.pvd", "w") as vtk:
#     vtk.write_mesh(Omega)
#     vtk.write_function(u_h)


Info    : Reading 'Omega_ex1_3.msh'...
Info    : 27 entities
Info    : 578 nodes
Info    : 256 elements
Info    : Done reading 'Omega_ex1_3.msh'
